In [1]:
import ccxt
import pandas as pd
import numpy as np
import time
import ta
import matplotlib.pyplot as plt
import joblib
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler

import time
import tensorflow as tf

In [2]:
exchange = ccxt.binance()

In [3]:
def fetch_ohlcv(symbol, exchange_id, timeframe, since=None, limit=1000, max_bars=None):
    exchange = getattr(ccxt, exchange_id)()
    exchange.load_markets()

    all_bars = []
    fetch_since = since if since else exchange.milliseconds() - (max_bars * 60 * 60 * 1000)

    while len(all_bars) < max_bars:
        try:
            new_bars = exchange.fetch_ohlcv(symbol, timeframe, since=fetch_since, limit=limit)

            all_bars.extend(new_bars)
            print(f"Fetched {len(new_bars)} bars, Total: {len(all_bars)}")

            fetch_since = new_bars[-1][0] + 1
            
            time.sleep(exchange.rateLimit / 1000)
            
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    df = pd.DataFrame(all_bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    df["SMA_20"] = ta.trend.sma_indicator(df["close"], window=20)  # 20-period SMA
    df["SMA_50"] = ta.trend.sma_indicator(df["close"], window=50)  # 50-period SMA
    df["EMA_20"] = ta.trend.ema_indicator(df["close"], window=20)  # 20-period EMA
    df["RSI_14"] = ta.momentum.rsi(df["close"], window=14)  # RSI
    df["MACD"] = ta.trend.macd(df["close"])  # MACD Line
    df["MACD_Signal"] = ta.trend.macd_signal(df["close"])  # MACD Signal Line
    df["ATR_14"] = ta.volatility.average_true_range(df["high"], df["low"], df["close"], window=14)  # ATR
    
    df.dropna(inplace=True)
    
    print(f"Final data count: {len(df)} bars")
    return df

In [4]:
model = tf.keras.models.load_model("btcpricetest_model2.keras")

scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")

In [5]:
feature_columns = ['open', 'high', 'low', 'volume', "SMA_20", "SMA_50", "EMA_20", "RSI_14", "MACD", "MACD_Signal", "ATR_14"]
seq_length = 50

In [6]:
from datetime import timedelta

def predict_next_hour():
    df = fetch_ohlcv("BTC/USDT", "binance", "1h", max_bars=100)

    df_feature = df[feature_columns]

    fetch = df_feature.iloc[-seq_length:]  

    last_timestamp = df["timestamp"].iloc[-1]
    next_timestamp = last_timestamp + timedelta(hours=1)

    df_scaled = scaler_X.transform(fetch)

    input_seq = df_scaled.reshape(1, seq_length, len(feature_columns))

    pred_scaled = model.predict(input_seq)

    pred_actual = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1))

    predicted_price = pred_actual[0][0]
    print(f"Predicted Timestamp: {next_timestamp}")
    print(f"Predicted BTC Price for Next Hour: ${predicted_price:.2f}")

    return next_timestamp, predicted_price

In [7]:
API_KEY  = "#"
API_SECRET  = "#"

In [8]:
exchange = ccxt.binance({
    'apiKey': API_KEY,
    'secret': API_SECRET,
    'options': {'defaultType': 'future'}
})

exchange.urls['api'] = exchange.urls['test']  # Override API endpoint for testnet (paper money)
exchange.set_sandbox_mode(True)

In [9]:
exchange.fetch_balance()

{'info': {'totalInitialMargin': '0.00000000',
  'totalMaintMargin': '0.00000000',
  'totalWalletBalance': '14781.85989126',
  'totalUnrealizedProfit': '0.00000000',
  'totalMarginBalance': '14781.85989126',
  'totalPositionInitialMargin': '0.00000000',
  'totalOpenOrderInitialMargin': '0.00000000',
  'totalCrossWalletBalance': '14781.85989126',
  'totalCrossUnPnl': '0.00000000',
  'availableBalance': '14781.85989126',
  'maxWithdrawAmount': '14781.85989126',
  'assets': [{'asset': 'FDUSD',
    'walletBalance': '0.00000000',
    'unrealizedProfit': '0.00000000',
    'marginBalance': '0.00000000',
    'maintMargin': '0.00000000',
    'initialMargin': '0.00000000',
    'positionInitialMargin': '0.00000000',
    'openOrderInitialMargin': '0.00000000',
    'crossWalletBalance': '0.00000000',
    'crossUnPnl': '0.00000000',
    'availableBalance': '0.00000000',
    'maxWithdrawAmount': '0.00000000',
    'updateTime': '0'},
   {'asset': 'BNB',
    'walletBalance': '0.00000000',
    'unrealize

In [10]:
def get_trade_amount(amount_usdt=5000): #Convert BTC to USDT for order amount
    ticker = exchange.fetch_ticker("BTC/USDT")
    btc_price = ticker["last"]
    trade_amount_btc = amount_usdt / btc_price 
    return trade_amount_btc

In [11]:
def get_trade_strategy():
    df = fetch_ohlcv("BTC/USDT", "binance", "1h", max_bars=55)

    next_timestamp, take_profit = predict_next_hour()

    exchange = ccxt.binance()
    ticker = exchange.fetch_ticker("BTC/USDT")
    entry_price = ticker["last"]
    
    if take_profit < entry_price:
        stop_loss = df["high"].iloc[-6:-1].max() # mean sell therefore stop_loss =  high
        order_type = "sell"
    else:
        stop_loss = df["low"].iloc[-6:-1].min() # mean buy therefore stop_loss =  low
        order_type = "buy"

    risk = abs(entry_price - stop_loss)
    reward = abs(take_profit - entry_price)
    risk_reward_ratio = reward / risk

    print("\n--- Trade Strategy ---")
    print(f"Entry Price (Current Price): ${entry_price:.2f}")
    print(f"Take Profit (TP): ${take_profit:.2f}")
    print(f"Stop Loss (SL): ${stop_loss:.2f}")
    print(f"Risk/Reward: {risk_reward_ratio:.2f}")
    print(f"Predicted Timeframe: {next_timestamp}")
    print(f"Order: {order_type}")
    
    if risk_reward_ratio < 1.75:
        print("Risk/Reward is lower than 1.75 no trade happen")
        return None  # **No trade execution**
    
    return {
        "entry_price": entry_price,
        "take_profit": take_profit,
        "stop_loss": stop_loss,
        "risk_reward_ratio": risk_reward_ratio,
        "predicted_timestamp": next_timestamp,
        "order_type": order_type
    }

In [12]:
def execute_trade_from_strategy(strategy):
    
    if strategy is None:
        print("No Trade happen")
        return None  
    
    symbol = "BTC/USDT"
    trade_amount = get_trade_amount(1000)    
    
    exchange.set_leverage(20, "BTC/USDT")
    
    entry_price = strategy["entry_price"]
    take_profit = strategy["take_profit"]
    stop_loss = strategy["stop_loss"]
    position_side = strategy["order_type"]

    entry_order = exchange.create_market_order(symbol, position_side, trade_amount)

    if position_side == "buy":
        tp_side = "sell"
    else:
        tp_side = "buy"
        
    tp_order = exchange.create_limit_order(symbol, tp_side, trade_amount, take_profit)

    sl_order = exchange.create_order(
        symbol = symbol,
        type = "STOP_MARKET",
        side = tp_side,
        amount = trade_amount,
        params = {"stopPrice": stop_loss}
    )
    
    return {
        "entry_order": entry_order,
        "tp_order": tp_order,
        "sl_order": sl_order
    }

In [13]:
trade_strategy = get_trade_strategy()  # Get trade data
trade_execution = execute_trade_from_strategy(trade_strategy)

Fetched 55 bars, Total: 55
Final data count: 6 bars
Fetched 100 bars, Total: 100
Final data count: 51 bars
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
Predicted Timestamp: 2025-03-17 12:00:00
Predicted BTC Price for Next Hour: $84449.25

--- Trade Strategy ---
Entry Price (Current Price): $83375.03
Take Profit (TP): $84449.25
Stop Loss (SL): $82682.62
Risk/Reward: 1.55
Predicted Timeframe: 2025-03-17 12:00:00
Order: buy
Risk/Reward is lower than 1.75 no trade happen
No Trade happen
